In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from py_stringmatching import  OverlapCoefficient
from py_stringmatching import  Dice
from py_stringmatching import  Jaccard

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kwgi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwgi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kwgi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Αρχικά θα διαβάσουμε τα αρχεία και θα τα μετατρέψουμε σε dataframes ελέγχοντας για null τιμές και τα μεγέθη των dataframes.Η κωδικοποίηση για το διάβασμα όλων των αρχείων θα είναι cp437 γιατί η utf8 μου έβγαζε error.

In [2]:
# load users and print first 5  users
users = pd.read_csv('BX-Users.csv',low_memory=False,sep=';',encoding='cp437')
print(users.isna().any())
print(users.shape)
users.head()

User-ID     False
Location    False
Age          True
dtype: bool
(278858, 3)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Παραπάνω δεν μας πειράζει που υπάρχουν null τιμές στην στήλη "Age".
Σε αυτό το σημείο χρειάζεται η υλοποίηση μιας μεθόδου που θα απομακρύνει κυρίως από τον τίτλο του βιβλίου κάποια σύμβολα που μπερδεύουν το pandas στην δημιουργία το dataframe books.Ο συνδυασμός συμβόλων επιλέχθηκαν εμπειρικά για να αφαιρεθούν.

In [3]:
# transforming file BX-Books.csv to the right format for csv and pandas
def transformCSV(inputfile,outputfile):
    with open(inputfile,encoding='cp437') as fp1,open(outputfile,'w',encoding='cp437') as fp2:
        line = fp1.readline()
        while line:
            if '&amp;' in line:
                line= line.replace('&amp;',' ')
            if ';\\' in line: 
                 line= line.replace(';\\',' ')
            if '"; ' in line: 
                 line= line.replace("'; \'",' ')  
            if ' ; '  in line: 
                 line= line.replace(' ; ',' ')  
            if '"; '  in line: 
                 line= line.replace('"; ',' ')  
            fp2.write(line)
            line = fp1.readline()

In [4]:
# transforming file BX-Book-Ratings.csv to the right format for csv and pandas
transformCSV('BX-Books.csv','BX-Books-r.csv')
# load books print first 5 books and print first 5 books
books = pd.read_csv('BX-Books-r.csv',error_bad_lines=False, low_memory=False,sep=';',encoding='cp437')
# get rid off unnecessary columns 
books = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication']]
print(books.shape)
print(books.isna().any())
books.head()

(271379, 4)
ISBN                   False
Book-Title             False
Book-Author             True
Year-Of-Publication    False
dtype: bool


,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999


Παραπάνω δεν μας πειράζει που υπάρχουν null τιμές στην στήλη "Book-Author".

In [5]:
# load ratings and print first 5 ratings 
ratings = pd.read_csv('BX-Book-Ratings.csv',low_memory=False,sep=';',encoding='cp437')
print(ratings.shape)
print(ratings.isna().any())
ratings.head()

(1149780, 3)
User-ID        False
ISBN           False
Book-Rating    False
dtype: bool


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Μέσω μελέτης των δεδομένων παρατήρησα πως υπάρχουν κάποια ratings για βιβλία που δεν υπάρχουν για αυτό το λόγο αφαιρώ αυτά τα ratings και ελέγχω αν συμβαίνει το ίδιο και για τους χρήστες.

In [6]:
# prune ratings with books that don't exist 
ratings = ratings.merge(books[['ISBN']], how = 'inner' , on = 'ISBN' )
print('Pruned ratings size: ' , ratings.shape)
# prune ratings with users that don't exist 
ratings = ratings.merge(users[['User-ID']], how = 'inner' , on = 'User-ID' )
print('Pruned ratings size: ' , ratings.shape)

Pruned ratings size:  (1031175, 3)
Pruned ratings size:  (1031175, 3)


Όπως φαίνεται η παραπάνω υπόθεση που κάναμε ισχύει μόνο για τα βιβλία. Στην συνέχεια αφαιρούνται όλα τα βιβλία που έχουν λιγότερες από 10 βαθμολογίες και όλους οι χρήστες που έχουν βαθμολογήσει λιγότερα από 5 βιβλία η διαδικασία επαναλαμβάνεται μέχρι να μην υπάρξει καμία αλλαγή στα σύνολα δεδομένων.

In [7]:
# intial sizes
ratings_size = ratings.shape
books_size = books.shape
users_size = users.shape

while True: 

    # prune all books that have less than 10 ratings  
    ratings_count = pd.DataFrame(ratings.groupby('ISBN')['Book-Rating'].count())
    ratings_count.rename(columns={'Book-Rating':'counts'},inplace = True)
    ratings_count.reset_index(inplace =True)
    ratings_count = ratings_count[ratings_count['counts'] > 10 ]
    # update ratings
    ratings = ratings.merge(ratings_count[['ISBN']], how = 'inner' , on = 'ISBN' )
    
    # prune all users that have less than 5 ratings 
    ratings_count = pd.DataFrame(ratings.groupby('User-ID')['Book-Rating'].count())
    ratings_count.rename(columns={'Book-Rating':'counts'},inplace = True)
    ratings_count.reset_index(inplace =True)
    ratings_count = ratings_count[ratings_count['counts'] > 5 ]
    # update ratings
    ratings = ratings.merge(ratings_count[['User-ID']], how = 'inner' , on = 'User-ID' )

    if ratings_size == ratings.shape and books_size == books.shape and users_size == users.shape: 
        break
    else:
        ratings_size = ratings.shape
        books_size = books.shape
        users_size = users.shape
        
# update dataframes
books = books.merge(ratings[['ISBN']], how = 'inner' , on = 'ISBN' ).drop_duplicates()
users = users.merge(ratings[['User-ID']],how = 'inner' , on = 'User-ID').drop_duplicates()    

# print new sizes of ratings,users and books  
print('ratings: ',ratings.shape)
print('users: ',users.shape)
print('books: ',books.shape)

ratings:  (385632, 3)
users:  (10404, 3)
books:  (12508, 4)


Όπως μπορείτε να δείτε, το σύνολο δεδομένων "ratings" μειώθηκε στις 385632 εγγραφές, το 'users' στις 10404 και το 'books' στις 12508.Παρακάτω υλοποιείτε μια μέθοδος για να παίρνουμε τον τίτλο του βιβλίου δίνοντας το ISBN του.

In [8]:
def getbooknamefromISBN(ISBN):
    return books.set_index('ISBN').loc[ISBN]['Book-Title']

In [9]:
getbooknamefromISBN(books['ISBN'].sample().tolist()[0])

'Zen Attitude'

Εδώ αρχίζει η προεπεξεργασία των τίτλων για την δημιουργία λέξεων κλειδιών. Με αυτόν τον κώδικα θέτουμε σε μια καινούρια στήλη "KeyWords" στο σύνολο δεδομένων books όλους τους τίτλους των βιβλίων με πεζά γράμματα.

In [10]:
books['KeyWords'] = books['Book-Title'].str.lower()
books['KeyWords'].head(10)

0                                  the kitchen god's wife
24                                          the testament
355                  beloved (plume contemporary fiction)
496     our dumb century: the onion presents 100 years...
511     new vegetarian: bold and beautiful recipes for...
526                                           wild animus
1748                                             airframe
1911                                             timeline
2197                                     prague : a novel
2215                                          lying awake
Name: KeyWords, dtype: object

Παρακάτω αφαιρούμε τα σημεία στίξης

In [11]:
# remove punctuations from Keywords(Title)
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [12]:
books["KeyWords"] = books["KeyWords"].apply(lambda x : remove_punctuation(x))
books["KeyWords"].head(10)

0                                   the kitchen gods wife
24                                          the testament
355                    beloved plume contemporary fiction
496     our dumb century the onion presents 100 years ...
511     new vegetarian bold and beautiful recipes for ...
526                                           wild animus
1748                                             airframe
1911                                             timeline
2197                                      prague  a novel
2215                                          lying awake
Name: KeyWords, dtype: object

Στην συνέχεια κάνουμε tokenize.

In [13]:
# tokenize Keywords(Title)
books["KeyWords"] = books["KeyWords"].fillna("").map(word_tokenize)
books["KeyWords"].head(10)

0                              [the, kitchen, gods, wife]
24                                       [the, testament]
355               [beloved, plume, contemporary, fiction]
496     [our, dumb, century, the, onion, presents, 100...
511     [new, vegetarian, bold, and, beautiful, recipe...
526                                        [wild, animus]
1748                                           [airframe]
1911                                           [timeline]
2197                                   [prague, a, novel]
2215                                       [lying, awake]
Name: KeyWords, dtype: object

Έπειτα αφαιρούμε τα stopwords στα Αγγλικά.

In [14]:
# remove stopwords from Keywords 
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [15]:
books["KeyWords"] = books["KeyWords"].apply(lambda x : remove_stopwords(x))
books["KeyWords"].head(10)

0                                   [kitchen, gods, wife]
24                                            [testament]
355               [beloved, plume, contemporary, fiction]
496     [dumb, century, onion, presents, 100, years, h...
511     [new, vegetarian, bold, beautiful, recipes, ev...
526                                        [wild, animus]
1748                                           [airframe]
1911                                           [timeline]
2197                                      [prague, novel]
2215                                       [lying, awake]
Name: KeyWords, dtype: object

Τέλος έχουμε την επιλογή να χρησιμοποιήσουμε είτε lemmatizer είτε stemmer για να κόψουμε τις καταλήξεις των λέξεων. Εγώ χρησιμοποιώ stemmer γιατί πιστεύω είναι ποιο χρήσιμος λόγο του μικρού μεγέθους των τίτλων.

In [16]:
# Instantiate lemmatizer 
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

In [17]:
# Instantiate Stemmer 
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

In [18]:
books["KeyWords"] = books["KeyWords"].apply(lambda x : word_stemmer(x))
books["KeyWords"].head(10)

0                                    [kitchen, god, wife]
24                                            [testament]
355                 [belov, plume, contemporari, fiction]
496     [dumb, centuri, onion, present, 100, year, hea...
511     [new, vegetarian, bold, beauti, recip, everi, ...
526                                         [wild, animu]
1748                                            [airfram]
1911                                            [timelin]
2197                                       [pragu, novel]
2215                                          [lie, awak]
Name: KeyWords, dtype: object

Στην συνέχεια υλοποιώ μια μέθοδο για να παίρνω τα 3 βιβλία με την υψηλότερη βαθμολογία για έναν χρήστη καθώς και όλα τα βιβλία που έχει βαθμολογήσει

In [19]:
# find 3 books' ISBN with the highest rating from the user and return them with all the books user has rated
def get3largest_values(user_id):
    user_books = ratings.set_index('User-ID').loc[user_id]
    user_books = user_books.reset_index()[['ISBN','Book-Rating']]
    return user_books ,user_books.nlargest(3,keep = "first", columns = ['Book-Rating'] ) 

In [20]:
user_books , fav_books = get3largest_values(users['User-ID'].sample().tolist()[0])
# print 10 books rated from user 
user_books["Book-Title"] = user_books["ISBN"].apply(lambda x : getbooknamefromISBN(x))
print(user_books.head(10))
# print favourite books 
print("Favourite books: ")
fav_books["Book-Title"] = fav_books["ISBN"].apply(lambda x : getbooknamefromISBN(x))
print(fav_books)

         ISBN  Book-Rating                                         Book-Title
0  0060987103            0  Wicked: The Life and Times of the Wicked Witch...
1  0393046974            0                              House of Sand and Fog
2  080410753X            0                             The Kitchen God's Wife
3  0060928336            0    Divine Secrets of the Ya-Ya Sisterhood: A Novel
4  0684857820            8                                              Lucky
5  1573229571            0                         About a Boy (Movie Tie-In)
6  0345339703            0  The Fellowship of the Ring (The Lord of the Ri...
7  0345443683            0   Blackwood Farm (Rice, Anne, Vampire Chronicles.)
8  0439139600           10       Harry Potter and the Goblet of Fire (Book 4)
9  0553213105            0                                Pride and Prejudice
Favourite books: 
          ISBN  Book-Rating                                    Book-Title
8   0439139600           10  Harry Potter and the 

Μέθοδος που δημιουργεί ένα “προφίλ” προτιμήσεων του χρήστη με βάση την
ένωση των λέξεων κλειδιών των 3 βιβλίων, τους συγγραφείς τους και το έτος
έκδοσης τους και επιστρέφει όλα αυτά και τα ISBN των βιβλίων που έχει βαθμολογήσει.

In [21]:
def make_user_profile(user_id):
    # get  readed and favourites books of user  
    user_books , fav_books = get3largest_values(user_id)
    
    # initialize 
    keywords = []
    authors = [] 
    years_of_publication = []
    
    for index,book in fav_books.iterrows():
            keywords += books.set_index('ISBN').loc[book['ISBN']]['KeyWords']
            authors.append(books.set_index('ISBN').loc[book['ISBN']]['Book-Author'].lower())
            years_of_publication.append(books.set_index('ISBN').loc[book['ISBN']]['Year-Of-Publication'])
    return keywords, authors, years_of_publication , user_books

In [22]:
profile = make_user_profile(users['User-ID'].sample().tolist()[0])
print(profile[0],profile[1],profile[2])
user_books  = profile[3]
user_books = user_books.reset_index()
user_books["Book-Title"] = user_books["ISBN"].apply(lambda x : getbooknamefromISBN(x))
user_books.head(10)

['redwal', 'redwal', 'book', '1', 'catcher', 'rye', 'confederaci', 'dunc'] ['brian jacques', 'j.d. salinger', 'john kennedy toole'] ['1998', '1991', '1995']


,index,ISBN,Book-Rating,Book-Title
0,0,0316769487,9,The Catcher in the Rye
1,1,0060928336,0,Divine Secrets of the Ya-Ya Sisterhood: A Novel
2,2,038079487X,5,What Looks Like Crazy On An Ordinary Day
3,3,0345443284,0,While I Was Gone
4,4,0441005489,10,"Redwall (Redwall, Book 1)"
5,5,0517122707,9,A Confederacy of Dunces
6,6,0060926201,7,"She Flew the Coop : A Novel Concerning Life, D..."
7,7,0743211995,0,An Hour Before Daylight : Memoirs of a Rural B...


H ομοιότητα Jaccard βασισμένη στον τύπο $ J(X,Y) = |X∩Y| / |X∪Y| $ 

In [23]:
def jaccard_similarity(x,y): 
    intersection_cardinality = len(set(x) & set(y))
    union_cardinality = len(set(x) | set(y))
    return intersection_cardinality/float(union_cardinality)

In [24]:
jaccard_similarity(["hallo","world"] ,["hello","world"])

0.3333333333333333

Έλεγχος για το αν υλοποιήθηκε σωστά.

In [25]:
j = Jaccard()
j.get_raw_score(["hallo","world"] ,["hello","world"])

0.3333333333333333

H ομοιότητα Dice coefficient βασισμένη στον τύπο $DSC(X,Y) = 2|X∩Y| / (|X|+|Y|)$

In [26]:
def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb)."""
    overlap = len(set(a) & set(b))
    return overlap * 2.0/(len(set(a)) + len(set(b)))

In [27]:
dice_coefficient(["hallo","world"] ,["hello","world"])

0.5

Έλεγχος για το αν υλοποιήθηκε σωστά 

In [28]:
dice = Dice()
dice.get_raw_score(["hallo","world"] ,["hello","world"])

0.5

Μέθοδος που υπολογίζει την ομοιότητα ενός βιβλίου με το “προφίλ” προτιμήσεων ενός χρήστη και ανάλογα το mode που θα επιλέξουμε επιστρέφει την ομοιότητα.(για mode = 1 η ομοιότητά υπολογίζεται ως εξής : Jaccard για τις λέξεις κλειδιά με βάρος 0.2, ισότητα για τους συγγραφείς 0.4,διαφορά ετών έκδοσης 0.4 και για mode = 2 : Dice coefficient για τις λέξεις κλειδιά με βάρος 0.5, ισότητα για τους συγγραφείς 0.3, διαφορά ετών έκδοσης 0.2)


In [29]:
def calc_similarity(user_profile,book,mode):
    # get keywords 
    profile_keywords = user_profile[0]
    book_keywords = book['KeyWords']
    
    # get author 
    profile_authors =  user_profile[1]
    book_author = book['Book-Author'].lower()
    
    # get years 
    profile_years =  user_profile[2]
    book_year = book['Year-Of-Publication']
    
    # calculate keywords_similarity 
    if mode == 1:
        keywords_similarity = jaccard_similarity(profile_keywords,book_keywords)
    elif mode == 2:
        keywords_similarity = dice_coefficient(profile_keywords,book_keywords)
    else:
        print("mode is missing or is incoreect")
        pass
    # calculate author similarity 
    if book_author in profile_authors:
        author_similarity = 1 
    else:
        author_similarity = 0
    
    # calculate years similarity
    year_similarity = min([1-(abs(int(x)-int(book_year))/2005) for x in profile_years])
    
    if mode == 1:
        return 0.2 * keywords_similarity + 0.4 * author_similarity + 0.4 * year_similarity
    elif mode==2:
        return 0.5 * keywords_similarity + 0.3 * author_similarity + 0.2 * year_similarity
    

In [30]:
# calculate similarity between profile that already have and random book
book = books.loc[books.sample().index.tolist()[0]]
print(book)
print("Similarity(mode=1) : ",calc_similarity(profile,book,1))
print("Similarity(mode=2) : ",calc_similarity(profile,book,2))

ISBN                    0812574621
Book-Title                 Animist
Book-Author            Eve Forward
Year-Of-Publication           2001
KeyWords                 [animist]
Name: 311802, dtype: object
Similarity(mode=1) :  0.3980049875311721
Similarity(mode=2) :  0.19900249376558604


Μέθοδος που προτείνει στον χρήστη τα 10 αντικείμενα (που δεν έχει βαθμολογήσει) τα οποία
είναι ομοιότερα με το προφίλ του, διατεταγμένα από το περισσότερο προς το
λιγότερο όμοιο.

In [78]:
def recommend(user_profile,mode):
    books_similarity = pd.DataFrame(books['ISBN'])
    str_similarity = ''
    if mode == 1 :
        str_similarity = 'Jaccard-Similarity'
    elif mode == 2 : 
        str_similarity = 'Dice-Similarity'
    else:
        print("mode is missing or is incoreect")
        pass
    books_similarity[str_similarity] = None   
    for index,book in books.iterrows():
        if book['ISBN'] not in user_profile[3]['ISBN'].unique():
            books_similarity.loc[index][str_similarity] =  calc_similarity(user_profile,book,mode)
    
    return books_similarity.sort_values([str_similarity],ascending=False).dropna().head(10)

In [82]:
# calculate and print recommended books with first mode  
recommended_books1 = recommend(profile,1)
recommended_books1["Book-Title"] = recommended_books1["ISBN"].apply(lambda x : getbooknamefromISBN(x))
recommended_books1

,ISBN,Jaccard-Similarity,Book-Title
112649,0802130208,0.872805,A Confederacy of Dunces (Evergreen Book)
376899,0316769177,0.855148,The Catcher in the Rye
367458,0394179692,0.853951,A confederacy of dunces
83071,0441000312,0.843646,"Salamandastron (Redwall, Book 5)"
292338,044100315X,0.843447,"The Bellmaker (Redwall, Book 7)"
114178,0441004164,0.843247,"Outcast of Redwall (Redwall, Book 8)"
80746,0380715309,0.843048,"Mattimeo (Redwall, Book 3)"
93676,0441005764,0.843048,"Mossflower (Redwall, Book 2)"
332493,0441006108,0.842848,"Mattimeo (Redwall, Book 3)"
235983,0380719223,0.842848,"Mariel of Redwall (Redwall, Book 4)"


In [83]:
# calculate and print recommended books with second mode  
recommended_books2 = recommend(profile,2)
recommended_books2["Book-Title"] = recommended_books2["ISBN"].apply(lambda x : getbooknamefromISBN(x))
recommended_books2

,ISBN,Dice-Similarity,Book-Title
112649,0802130208,0.77163,A Confederacy of Dunces (Evergreen Book)
376899,0316769177,0.721225,The Catcher in the Rye
367458,0394179692,0.720626,A confederacy of dunces
83071,0441000312,0.681419,"Salamandastron (Redwall, Book 5)"
292338,044100315X,0.681319,"The Bellmaker (Redwall, Book 7)"
114178,0441004164,0.68122,"Outcast of Redwall (Redwall, Book 8)"
93676,0441005764,0.68112,"Mossflower (Redwall, Book 2)"
80746,0380715309,0.68112,"Mattimeo (Redwall, Book 3)"
235983,0380719223,0.68102,"Mariel of Redwall (Redwall, Book 4)"
332493,0441006108,0.68102,"Mattimeo (Redwall, Book 3)"


Εδώ ξεκινάει το πείραμα. Επιλέγουμε τυχαία 5 χρήστες και παράγουμε για
αυτούς τις σχετικές προτάσεις και με τις δύο εναλλακτικές, οι οποίες και θα
καταγράφουν σε αντίστοιχα αρχεία εξόδου (διαφορετικά).Άμα θέλετε μπορείτε να παραλείψετε την δημιουργία προβλέψεων και να διαβάσετε τις υπάρχοντες προβλέψεις (άμα υπάρχουν).  


In [84]:
def make_predictions():
    # random sample of 5 users 
    sampled_users = users['User-ID'].sample(5).tolist()

    for user in sampled_users:
        # make user's profile 
        profile = make_user_profile(user)

        # recommend 10 books with first method
        recommended_books1 = recommend(profile,1)
        recommended_books1["Book-Title"] = recommended_books1["ISBN"].apply(lambda x : getbooknamefromISBN(x))

        # recommend 10 books with the second method
        recommended_books2 = recommend(profile,2)
        recommended_books2["Book-Title"] = recommended_books2["ISBN"].apply(lambda x : getbooknamefromISBN(x))

        # save them 
        recommended_books1.to_csv(str(user)+'_1.csv',index= None,encoding='cp437')
        recommended_books2.to_csv(str(user)+'_2.csv',index= None,encoding='cp437')
    
    # save sampled users 
    with open('sampled_users.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(sampled_users)
        

In [85]:
make_predictions()

Για κάθε χρήστη που τυχαία επέλεξε προηγουμένως, μετραει την ομοιότητα των λιστών με τα αποτέλεσματα
υπολογίζοντας τη μέση επικάλυψη τους και τα επιστρέφει μαζί με 2 λίστες που έχουν τα αποτελέσματα των χρηστών.Πρωτα πρέπει να φτιαχτεί μια μέθοδος για να μετράμε την επικάλυψη βάση του τύπου $overlap_coefficient(X,Y)=|X∩Y|/min(|X|,|Y|)$

In [36]:
def overlap(x,y):
    intersection_cardinality = len(set(x) & set(y))
    return intersection_cardinality / min(len(set(x)),len(set(y)))

In [37]:
overlap(["hallo","world"] ,["hello","world"])

0.5

Έλεγχος για το αν υλοποιήθηκε σωστά. 

In [38]:
oc = OverlapCoefficient()
oc.get_raw_score(["hallo","world"] ,["hello","world"])

0.5

In [86]:
def read_sampled_users():
    sampled_users = [] 
    with open('sampled_users.csv', 'r') as f:
        reader = csv.reader(f)
        sampled_users = list(reader)[0]
    return sampled_users

In [87]:
# read the User-ID fro sampled users  
sampled_users = read_sampled_users()
print(sampled_users)

['101381', '226743', '271538', '162626', '102702']


In [51]:
def read_recommended_books(sampled_users):
    # lists with recommended books for every user 
    recommended_books1_list = []
    recommended_books2_list = [] 
    
    for user in sampled_users:
        # read recommended_books with first method
        recommended_books1 = pd.read_csv(str(user) + '_1.csv',low_memory=False,encoding='cp437',dtype={'ISBN': str})
        recommended_books1_list.append(recommended_books1)
        # read recommended_books with second method
        recommended_books2 = pd.read_csv(str(user) + '_2.csv',low_memory=False,encoding='cp437',dtype={'ISBN': str})
        recommended_books2_list.append(recommended_books2)
    
    return recommended_books1_list,recommended_books2_list

In [112]:
# read reconmended books
recommended_books1_list , recommended_books2_list = read_recommended_books(sampled_users)
# print first sampled user recommended books 
print(recommended_books1_list[0])
print(recommended_books2_list[0])

         ISBN  Jaccard-Similarity                         Book-Title
0  0316693642            0.872805                Along Came a Spider
1  0446606189            0.858803    Cat   Mouse (Alex Cross Novels)
2  0446604801            0.858603    Jack   Jill (Alex Cross Novels)
3  0446605484            0.858404  Roses Are Red (Alex Cross Novels)
4  0812969812            0.848005                Blessings : A Novel
5  0375502238            0.823204                          Blessings
6  0061015725            0.823005                               Prey
7  0375500510            0.818803           Black and Blue : A Novel
8  0316693715            0.818404             Jack and Jill: A Novel
9  0316666009            0.818404                1st to Die: A Novel
         ISBN  Dice-Similarity                         Book-Title
0  0316693642         0.771630                Along Came a Spider
1  0446606189         0.730171    Cat   Mouse (Alex Cross Novels)
2  0446604801         0.730071    Jack   Ji

In [113]:
def calc_weighted_overlap(recommended_books1,recommended_books2):
    weighted_overlap = 0
    sum_overlap = 0 
    for i in range(10):
        
        # get isbn
        isbns1 = recommended_books1.iloc[0:i+1, 0].tolist()
        isbns2 = recommended_books2.iloc[0:i+1, 0].tolist()
        
        # update sum overlap 
        sum_overlap += overlap(isbns1,isbns2)
            
    # compute weighted_overlap
    weighted_overlap = sum_overlap/10

    return weighted_overlap

In [114]:
# calculate weighted_overlaps for every user  
weighted_overlaps = []       

for i in range(5):    
    weighted_overlap = calc_weighted_overlap(recommended_books1_list[i],recommended_books2_list[i])
    weighted_overlaps.append(weighted_overlap)
    
print('Weighted_Overlaps')
for i in range(5):
    print('User: ',sampled_users[i]," ",weighted_overlaps[i])

Weighted_Overlaps
User:  101381   1.0
User:  226743   0.9646031746031746
User:  271538   1.0
User:  162626   0.9746031746031745
User:  102702   0.975


Για κάθε χρήστη, υπολογίζει σε πόσες λίστες εμφανίζεται το κάθε αποτέλεσμα και
ταξινομεί τα σε μία νέα λίστα—πρώτα με βάση του πλήθους εμφανίσεων και
μετά με βάση την ομοιότητα με το προφίλ του χρήστη. H ομοιότητα που χρησιμοποιείται στην ταξινόμηση είναι ο μ.ο. των Jaccard και Dice και όπου δεν υπάρχει μια εκ των δυο υπολογίζεται.


Μέθοδος δημιουργίας golden standard δωσμένου μιας λίστας ενος χρήστη και την λίστα με τις λίστες όλων των χρηστών.

In [115]:
def make_golden_standard(recommended_books1,recommended_books2,user_id):
    # change name to similarity to 
    golden_standard = pd.merge(recommended_books1, recommended_books2, on = ['ISBN','Book-Title'], how = 'outer')
    golden_standard['counts'] = None 
    
    user_profile = make_user_profile(user_id)
    
    # check if has only jaccard or dice similarity or both ,update accordingly the counts and the similarities 
    for index,book in golden_standard.iterrows():
        if pd.isna(book['Jaccard-Similarity'])  : 
            golden_standard.at[index,'counts'] = 1
            golden_standard.at[index,'Jaccard-Similarity'] = calc_similarity(user_profile,books.set_index('ISBN').loc[book['ISBN']],1)
        elif pd.isna(book['Dice-Similarity'])  : 
            golden_standard.at[index,'counts'] = 1
            golden_standard.at[index,'Dice-Similarity'] = calc_similarity(user_profile,books.set_index('ISBN').loc[book['ISBN']],2)
        else:
            golden_standard.at[index,'counts'] = 2
   
    # calculate weighted similarity 
    golden_standard['Weighted-Similarity'] =  (golden_standard['Dice-Similarity'] + golden_standard['Jaccard-Similarity'])/2
    golden_standard.sort_values(by=['counts','Weighted-Similarity'],ascending=False,inplace=True)
    
    #save golden standard 
    golden_standard.to_csv('golden' + str(user_id) +'.csv',index= None,encoding='cp437')
    
    return golden_standard

Eπανάλάψη του βήματος 2 συγκρίνει όμως κάθε λίστα
με την golden_standard που παράχθηκε (μέχρι το 10 της στοιχείο).

In [120]:
# make golden standard for every user and calculate weighted overlap  
weighted_overlaps1 = [] 
weighted_overlaps2 = [] 
for i in range(5):    
    golden_standard = make_golden_standard(recommended_books1_list[i],recommended_books2_list[i],int(sampled_users[i]))
    # weighted overlap for first list 
    weighted_overlap = calc_weighted_overlap(golden_standard.head(10),recommended_books1_list[i])
    weighted_overlaps1.append(weighted_overlap)
    
    # weighted overlap for second list
    weighted_overlap = calc_weighted_overlap(golden_standard.head(10),recommended_books2_list[i])
    weighted_overlaps2.append(weighted_overlap)
    
print('Weighted_Overlaps(Jaccard-Golden)')
for i in range(5):
    print('User: ',sampled_users[i]," ",weighted_overlaps1[i])
    
print('\nWeighted_Overlaps(Dice-Golden)')
for i in range(5):
    print('User: ',sampled_users[i]," ",weighted_overlaps2[i])

Weighted_Overlaps(Jaccard-Golden)
User:  101381   1.0
User:  226743   0.9888888888888889
User:  271538   1.0
User:  162626   1.0
User:  102702   1.0

Weighted_Overlaps(Dice-Golden)
User:  101381   1.0
User:  226743   0.9757142857142858
User:  271538   1.0
User:  162626   0.9746031746031745
User:  102702   0.975


In [109]:
prof = make_user_profile(int(sampled_users[4]))
overlap(prof[3]['ISBN'],recommended_books1_list[4]['ISBN'])

0.0

In [110]:
prof

(['harri',
  'potter',
  'sorcer',
  'stone',
  'harri',
  'potter',
  'paperback',
  'harri',
  'potter',
  'chamber',
  'secret',
  'book',
  '2',
  'deep',
  'midnight'],
 ['j. k. rowling', 'j. k. rowling', 'shannon drake'],
 ['1999', '1999', '2001'],
            ISBN  Book-Rating
 0    0446605484            0
 1    0312974485            0
 2    0446608955            8
 3    059035342X           10
 4    0440213525            0
 ..          ...          ...
 165  0758205481           10
 166  0373273134           10
 167  084394952X           10
 168  0821758578           10
 169  050552113X           10
 
 [170 rows x 2 columns])

In [118]:
recommended_books1_list[1]

,ISBN,Jaccard-Similarity,Book-Title
0,0385335482,0.856943,Confessions of a Shopaholic (Summer Display Op...
1,0440241812,0.845555,Shopaholic Takes Manhattan
2,0446530522,0.830570,Cane River
3,0446525502,0.814986,The Rescue
4,0440241413,0.813887,Confessions of a Shopaholic
5,0385336179,0.812934,Shopaholic Ties the Knot
6,0446527785,0.799800,A Bend in the Road
7,0446611867,0.799800,A Bend in the Road
8,0446691798,0.799601,Nights in Rodanthe
9,0446608955,0.799601,A Walk to Remember


In [117]:
recommended_books2_list[1]

,ISBN,Dice-Similarity,Book-Title
0,0385335482,0.722122,Confessions of a Shopaholic (Summer Display Op...
1,0440241812,0.687201,Shopaholic Takes Manhattan
2,0446530522,0.633234,Cane River
3,0446525502,0.571229,The Rescue
4,0440241413,0.566467,Confessions of a Shopaholic
5,0385336179,0.562300,Shopaholic Ties the Knot
6,0446611867,0.499900,A Bend in the Road
7,0446527785,0.499900,A Bend in the Road
8,0446608955,0.499800,A Walk to Remember
9,0446532452,0.499800,The Wedding
